In [3]:
import os
import pandas as pd
import numpy as np

sub_path = "../iceberg_code/ans"
all_files = os.listdir(sub_path)

# Read and concatenate submissions
outs = [pd.read_csv(open(os.path.join(sub_path, f)), index_col=0) for f in all_files]
concat_sub = pd.concat(outs, axis=1)
cols = list(map(lambda x: "is_iceberg_" + str(x), range(len(concat_sub.columns))))
concat_sub.columns = cols
concat_sub.reset_index(inplace=True)
concat_sub.head()

,id,is_iceberg_0,is_iceberg_1,is_iceberg_2,is_iceberg_3,is_iceberg_4,is_iceberg_5,is_iceberg_6,is_iceberg_7,is_iceberg_8,...,is_iceberg_11,is_iceberg_12,is_iceberg_13,is_iceberg_14,is_iceberg_15,is_iceberg_16,is_iceberg_17,is_iceberg_18,is_iceberg_19,is_iceberg_20
0,5941774d,0.034383,0.103063,0.315171,0.110608,0.999992,0.158412,0.040524,0.60002,0.001817,...,0.222085,0.039642,0.774446,0.036656,0.331437,0.013640,0.397247,0.000003,0.257807,0.050374
1,4023181e,0.001077,0.709638,0.467635,0.485623,0.000810,0.423263,0.441693,0.76029,0.000006,...,0.238603,0.111164,0.002052,0.131221,0.246653,0.222334,0.342772,0.391367,0.275332,0.061725
2,b20200e4,0.000000,0.000000,0.043588,0.000000,0.000002,0.000542,0.000132,0.00001,0.000000,...,0.000967,0.000341,0.004832,0.000137,0.000409,0.000040,0.036616,0.000092,0.000012,0.000075
3,e7f018bb,0.994085,0.975599,0.999919,0.979755,1.000000,0.997309,0.996459,0.99296,0.999997,...,0.238644,0.998179,0.987940,0.996498,0.996336,0.963253,0.894716,0.981080,0.945461,0.999927
4,4371c8c3,0.000003,0.066838,0.142250,0.255516,0.000045,0.004247,0.612491,0.50877,0.000807,...,0.028673,0.021845,0.003671,0.021123,0.012114,0.003375,0.332533,0.005542,0.033289,0.064295


In [4]:
concat_sub['is_iceberg_max'] = concat_sub.iloc[:, 1:6].max(axis=1)
concat_sub['is_iceberg_min'] = concat_sub.iloc[:, 1:6].min(axis=1)
concat_sub['is_iceberg_mean'] = concat_sub.iloc[:, 1:6].mean(axis=1)
concat_sub['is_iceberg_median'] = concat_sub.iloc[:, 1:6].median(axis=1)

## Mean stack

In [5]:
concat_sub['is_iceberg'] = concat_sub['is_iceberg_mean']
concat_sub[['id', 'is_iceberg']].to_csv('stack_mean.csv', 
                                        index=False, float_format='%.6f')

## Median stack

In [6]:
concat_sub['is_iceberg'] = concat_sub['is_iceberg_median']
concat_sub[['id', 'is_iceberg']].to_csv('stack_median.csv', 
                                        index=False, float_format='%.6f')

In [7]:
# set up cutoff threshold for lower and upper bounds, easy to twist 
cutoff_lo = 0.8
cutoff_hi = 0.2

## PushOut + Median Stacking

In [8]:
concat_sub['is_iceberg'] = np.where(np.all(concat_sub.iloc[:,1:6] > cutoff_lo, axis=1), 1, 
                                    np.where(np.all(concat_sub.iloc[:,1:6] < cutoff_hi, axis=1),
                                             0, concat_sub['is_iceberg_median']))
concat_sub[['id', 'is_iceberg']].to_csv('stack_pushout_median.csv', 
                                        index=False, float_format='%.6f')

## MinMax + Mean Stacking

In [9]:
concat_sub['is_iceberg'] = np.where(np.all(concat_sub.iloc[:,1:6] > cutoff_lo, axis=1), 
                                    concat_sub['is_iceberg_max'], 
                                    np.where(np.all(concat_sub.iloc[:,1:6] < cutoff_hi, axis=1),
                                             concat_sub['is_iceberg_min'], 
                                             concat_sub['is_iceberg_mean']))
concat_sub[['id', 'is_iceberg']].to_csv('stack_minmax_mean.csv', 
                                        index=False, float_format='%.6f')

## MinMax + Median Stacking

In [10]:
concat_sub['is_iceberg'] = np.where(np.all(concat_sub.iloc[:,1:6] > cutoff_lo, axis=1), 
                                    concat_sub['is_iceberg_max'], 
                                    np.where(np.all(concat_sub.iloc[:,1:6] < cutoff_hi, axis=1),
                                             concat_sub['is_iceberg_min'], 
                                             concat_sub['is_iceberg_median']))
concat_sub[['id', 'is_iceberg']].to_csv('stack_minmax_median.csv', 
                                        index=False, float_format='%.6f')

## MinMax + BestBase Stacking

In [11]:
# load the model with best base performance
sub_base = pd.read_csv('../iceberg_code/best_answer.csv')

concat_sub['is_iceberg_base'] = sub_base['is_iceberg']
concat_sub['is_iceberg'] = np.where(np.all(concat_sub.iloc[:,1:6] > cutoff_lo, axis=1), 
                                    concat_sub['is_iceberg_max'], 
                                    np.where(np.all(concat_sub.iloc[:,1:6] < cutoff_hi, axis=1),
                                             concat_sub['is_iceberg_min'], 
                                             concat_sub['is_iceberg_base']))
concat_sub[['id', 'is_iceberg']].to_csv('stack_minmax_bestbase.csv', 
                                        index=False, float_format='%.6f')


FileNotFoundError: File b'../iceberg_code/best_answer.csv' does not exist